In [ ]:
import time
import re
import numpy as np
import pandas as pd
import pathlib
# define rooms and items

#furniture
couch = {
    "name": "couch",
    "type": "furniture",
}

double_bed = {
    "name": "double bed",
    "type": "furniture",
}

dresser = {
    "name": "dresser",
    "type": "furniture",
}

dining_table = {
    "name": "dining table",
    "type": "furniture",
}

queen_bed = {
    "name": "queen bed",
    "type": "furniture",
}

piano = {
    "name": "piano",
    "type": "furniture",
}

#doors
door_a = {
    "name": "door a",
    "type": "door",
}

door_b = {
    "name": "door b",
    "type": "door",
}

door_b = {
    "name": "door b",
    "type": "door",
}

door_c = {
    "name": "door c",
    "type": "door",
}

door_d = {
    "name": "door d",
    "type": "door",
}

#keys
key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}

key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}


key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c,
}

key_d = {
    "name": "key for door d",
    "type": "key",
    "target": door_d,
}

# rooms

game_room = {
    "name": "game room",
    "type": "room",
}

bedroom_1 = {
    "name": "bedroom one",
    "type": "room",
}

bedroom_2 = {
    "name": "bedroom two",
    "type": "room",
}

living_room = {
    "name": "living room",
    "type": "room",
}

outside = {
  "name": "outside"
}

all_rooms = [game_room, bedroom_1, bedroom_2, living_room, outside]

all_doors = [door_a, door_b, door_c, door_d]

# define which items/rooms are related

object_relations = {
    "game room": [couch, piano, door_a],
    "bedroom one": [queen_bed, door_a, door_b, door_c],
    "bedroom two": [double_bed, dresser, door_b],
    "living room": [dining_table, door_c, door_d],
    "queen bed": [key_b],
    "piano": [key_a],
    "double bed" : [key_c],
    "dresser" : [key_d],
    "outside": [door_d],
    "door a": [game_room, bedroom_1],
    "door b": [bedroom_1, bedroom_2],
    "door c": [bedroom_1, living_room],
    "door d": [living_room, outside],
}

# define game state. Do not directly change this dict. 
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This 
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside,
    "start_time": time.time()
}

#functions
def hof(time_elapsed):
  
    #pathlib.Path().absolute()
    df = pd.read_csv('HOF.csv')
    columns = list(df.columns)
    
    df2 = pd.DataFrame([[name,time_elapsed]],columns=columns)

    df2 = pd.concat([df,df2])
    df2.sort_values(by=['Time (sec.)'], inplace = True)

    pd.DataFrame.to_csv(df2, 'HOF.csv', index = False)

    df2.index = range(1,len(df2.index)+1)
    print("\nHall of Fame\n")
    print(df2.head(10))

    return

def game_parameters():
    max_time = 'x'

    while re.findall(r'[^0-9*]',max_time) != []:    
        max_time = input ('What is the time limit (in seconds) to escape the Room (enter a number between 30 and 600)?')
 
    max_time = int (max_time)

    if max_time < 30:
        max_time = 30
        print('Your time limit is 30 seconds')
    elif max_time > 600:
        max_time = 600
        print('Your time limit is 600 seconds')
    else:
        print('Your time limit is %d' %max_time)
    
    return max_time

def linebreak():
    """
    Print a line break
    """
    print("\n")

def start_game():
    """
    Start the game
    """
    print("You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!")
    linebreak()
    play_room(game_state["current_room"])
    linebreak()
        
def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either 
    explore (list all items in this room) or examine an item found here.
    """
    
    current_time = time.time()
    elapsed_time = current_time - game_state["start_time"]
    print("Elapsed_time: %d seconds have already passed" %round(elapsed_time))
    linebreak()
    if elapsed_time > max_time:
        print("Your time is up. GAME OVER")
        return
    
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print("Congrats! You escaped the room!")
        hof(elapsed_time)
    else:
        print("You are now in " + room["name"])
        intended_action = input("What would you like to do? Type 'explore' or 'examine'?").lower().strip()
        if intended_action == "explore":
            explore_room(room)
            play_room(room)
        elif intended_action == "examine":
            examine_item(input("What would you like to examine?").lower().strip())
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'.")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))
    linebreak()

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def enigma(door):
    if door == "door a":
        print("A bottle and the cork costs $1.10 The bottle is $1.00 more expensive than the cork. How much is the cork?")
        answer_a = input("Please enter your answer to open the door:")

        while answer_a.strip() != "0.05":
            answer_a = input("Try again! Please enter your answer to open the door:")
        else:
            return
        
    elif door == "door b":
        print("In a bicycle race, the man who came two places in front of the last man finished one ahead of the man who came 5th. How many contestants were there?")
        answer_b = input("Please enter your answer to open the door:")

        while (answer_b.strip() != "6" and answer_b.lower().strip() != "six"):
            answer_b = input("Try again! Please enter your answer to open the door:")
        else:
            return
              
    elif door == "door c":
        print("Mr. Smith has 4 daughters. Each of his daughters has a brother. How many children does Mr. Smith have?")
        answer_c = input("Please enter your answer to open the door:")
        
        while (answer_c.strip() != "5" and answer_c.lower().strip() != "five"):
            answer_c = input("Try again! Please enter your answer to open the door:")
        else:
            return
        
    else:
        print("How many days are there in 4 years?")
        answer_d = input("Please enter your answer to open the door:")
        
        while answer_d.strip() != "1461":
            answer_d = input("Try again! Please enter your answer to open the door:")
        else:
            return
    linebreak()

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been 
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None
    
    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "You examine " + item_name + ". "
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    linebreak()
                    print("You unlocked it with a key but to open it you need to solve an enigma.")
                    enigma(item["name"])
                    output = "You solved the enigma and the door opened"
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find " + item_found["name"] + "."
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break
    linebreak()

    if(output is None):
        print("The item you requested is not found in the current room.")
    if(next_room and input("Do you want to go to the next room? Enter 'yes' or 'no'").lower().strip() == 'yes'):
        play_room(next_room)
    else:
        play_room(current_room)

    linebreak()
    
#start game
game_state = INIT_GAME_STATE.copy()
name = input ('Hello!! What´s your name?')
max_time = game_parameters()
linebreak()
game_state["start_time"]= time.time()
start_game()